In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


In [3]:
!nvidia-smi

Wed Nov  2 17:48:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    38W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from align_faces import warp_and_crop_face, get_reference_facial_points

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
from torchmetrics import F1Score

import time

In [5]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [6]:
train_dir_path = '../input/data/train/'
train_image_path = '../input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
exp_train = pd.read_csv(train_dir_path+'expanded_train.csv')

def get_age_range(age):
    if age < 30:
        return 0
    elif 30 <= age < 60:
        return 1
    else:
        return 2
dt_train['age_range'] = dt_train['age'].apply(lambda x : get_age_range(x))
dt_train

,id,gender,race,age,path,age_range
0,1,female,Asian,45,000001_female_Asian_45,1
1,2,female,Asian,52,000002_female_Asian_52,1
2,4,male,Asian,54,000004_male_Asian_54,1
3,5,female,Asian,58,000005_female_Asian_58,1
4,6,female,Asian,59,000006_female_Asian_59,1
...,...,...,...,...,...,...
2695,6954,male,Asian,19,006954_male_Asian_19,0
2696,6955,male,Asian,19,006955_male_Asian_19,0
2697,6956,male,Asian,19,006956_male_Asian_19,0
2698,6957,male,Asian,20,006957_male_Asian_20,0


In [7]:
train_idx, valid_idx = train_test_split(np.arange(len(dt_train)),
                                       test_size=0.2,
                                       shuffle=True,
                                       stratify=dt_train['age_range'])
dt_train.iloc[train_idx].head(70)

,id,gender,race,age,path,age_range
690,1396,male,Asian,45,001396_male_Asian_45,1
374,829,female,Asian,55,000829_female_Asian_55,1
268,675,female,Asian,59,000675_female_Asian_59,1
1394,3431,female,Asian,50,003431_female_Asian_50,1
1956,4487,male,Asian,18,004487_male_Asian_18,0
...,...,...,...,...,...,...
1156,3108,female,Asian,19,003108_female_Asian_19,0
1682,3863,female,Asian,56,003863_female_Asian_56,1
42,66,female,Asian,53,000066_female_Asian_53,1
2194,5487,female,Asian,48,005487_female_Asian_48,1


In [8]:
exp_train.drop(columns=['Unnamed: 0', 'Has_Face', 'Score', 'LE_X', 'LE_Y', 'RE_X', 'RE_Y', 'N_X', 'N_Y', 'LM_X', 'LM_Y', 'RM_X', 'RM_Y'], inplace=True)

In [9]:
def make_new_path(x):
    return x.split("images")[0] + 'new_images' + x.split("images")[1]

In [10]:
exp_train['new_path'] = exp_train['Path'].apply(make_new_path)

In [11]:
exp_train

,PersonID,Path,Filename,Class,Mask,Gender,Age,Age_Class,BBoxX1,BBoxY1,BBoxX2,BBoxY2,new_path
0,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,110,146,250,347,../input/data/train/new_images/000001_female_A...
1,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,112,160,245,334,../input/data/train/new_images/000001_female_A...
2,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,107,127,230,311,../input/data/train/new_images/000001_female_A...
3,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,120,143,244,326,../input/data/train/new_images/000001_female_A...
4,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,10,1,1,45,1,122,125,244,316,../input/data/train/new_images/000001_female_A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,116,181,246,340,../input/data/train/new_images/006959_male_Asi...
18896,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,119,182,260,354,../input/data/train/new_images/006959_male_Asi...
18897,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,6,1,0,19,0,111,180,244,349,../input/data/train/new_images/006959_male_Asi...
18898,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,12,2,0,19,0,112,178,255,364,../input/data/train/new_images/006959_male_Asi...


In [12]:
split_exp_train = exp_train[exp_train['PersonID'].isin(list(dt_train.iloc[train_idx]["id"]))]
split_exp_valid = exp_train[exp_train['PersonID'].isin(list(dt_train.iloc[valid_idx]["id"]))]
print(f"index size: {len(train_idx)} == file estimate: {len(train_idx) * 7} == split size: {len(split_exp_train)}")
split_exp_train

index size: 2160 == file estimate: 15120 == split size: 15120


,PersonID,Path,Filename,Class,Mask,Gender,Age,Age_Class,BBoxX1,BBoxY1,BBoxX2,BBoxY2,new_path
0,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,110,146,250,347,../input/data/train/new_images/000001_female_A...
1,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,112,160,245,334,../input/data/train/new_images/000001_female_A...
2,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,107,127,230,311,../input/data/train/new_images/000001_female_A...
3,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,4,0,1,45,1,120,143,244,326,../input/data/train/new_images/000001_female_A...
4,1,../input/data/train/images/000001_female_Asian...,../input/data/train/images/000001_female_Asian...,10,1,1,45,1,122,125,244,316,../input/data/train/new_images/000001_female_A...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,116,181,246,340,../input/data/train/new_images/006959_male_Asi...
18896,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,0,0,0,19,0,119,182,260,354,../input/data/train/new_images/006959_male_Asi...
18897,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,6,1,0,19,0,111,180,244,349,../input/data/train/new_images/006959_male_Asi...
18898,6959,../input/data/train/images/006959_male_Asian_1...,../input/data/train/images/006959_male_Asian_1...,12,2,0,19,0,112,178,255,364,../input/data/train/new_images/006959_male_Asi...


In [ ]:
# for idx in range(len(exp_train)):
#     filename = exp_train.iloc[idx]['Path']
#     new_filename = exp_train.iloc[idx]['new_path']
#     parent_dir = '/'.join(new_filename.split("/")[:-1])
    
#     if not os.path.exists(parent_dir):
#         os.makedirs(parent_dir)
    
#     X = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
#     x1 = exp_train.iloc[idx]['BBoxX1']
#     x2 = exp_train.iloc[idx]['BBoxX2']
#     y1 = exp_train.iloc[idx]['BBoxY1']
#     y2 = exp_train.iloc[idx]['BBoxY2']
#     w = x2 - x1
#     h = y2 - y1
#     w = w // 3
#     h = h // 3

#     nx1 = max(0, x1-w)
#     nx2 = min(384, x2+w)
#     ny1 = max(0, y1-h)
#     ny2 = min(512, y2+h)
#     X = X[ny1:ny2, nx1:nx2]
    
    
#     X = cv2.resize(X, (350, 350))
#     if idx % 3000 == 0:
#         plt.imshow(X)
#         plt.show()
        
#     X = cv2.cvtColor(X, cv2.COLOR_RGB2BGR)
    
#     cv2.imwrite(new_filename, X)

In [13]:
train_image = split_exp_train.loc[:,"new_path"]
train_label = split_exp_train.loc[:,"Class"]

valid_image = split_exp_valid.loc[:,"new_path"]
valid_label = split_exp_valid.loc[:,"Class"]

In [14]:
print(len(train_image))
print(len(valid_image))

15120
3780


In [15]:
train_data = pd.Series(train_image)
train_label = pd.Series(train_label)

valid_data = pd.Series(valid_image)
valid_label = pd.Series(valid_label)

In [16]:
from torchvision.transforms import Resize, ToTensor, Normalize, Compose, CenterCrop, ColorJitter
from PIL import Image

class Dataset_Mask(Dataset):
    def __init__(self, data, label, encoding=True, midcrop=True, transform=None, is_train=True):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = data.reset_index(drop=True)
        self.is_train = is_train
        self.label = label.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.transform:
            return self.transform(X), y

        return X, y

In [17]:
mask_train_set = Dataset_Mask(data=train_data, label=train_label, is_train=True, transform=transforms.Compose([
                                    ToTensor(),
                            ]))

In [18]:
mask_val_set = Dataset_Mask(data=valid_data, label=valid_label, is_train=False, transform = transforms.Compose([
                                    ToTensor(),
                            ]))

In [19]:
batch_size = 64

train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, num_workers=2, drop_last=True)
val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, num_workers=2)

In [20]:
model = torchvision.models.resnet50(pretrained=True)
# print('필요 입력 채널 개수', model.conv1.weight.shape[1])
# print('네트워크 출력 채널 개수', model.fc.weight.shape[0])
# print(model)

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
import math
class_num = 18
model.fc = nn.Linear(in_features=2048, out_features=class_num, bias=True)
nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

print('필요 입력 채널 개수', model.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', model.fc.weight.shape[0])

필요 입력 채널 개수 3
네트워크 출력 채널 개수 18


In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model = torch.nn.DataParallel(model)
model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 50

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

#lr = 1e-3
#betas = (0.9, 0.999)
#weight_decay = 0.5e-4
#eps = 1e-8
#optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay, eps=eps)

using cuda:0


In [23]:
np.set_printoptions(precision=3)
n_param = 0
for p_idx, (param_name, param) in enumerate(model.named_parameters()):
    if param.requires_grad:
        
        #if param_name.startswith('module.fc') or param_name.startswith('module.layer4') or param_name.startswith('module.layer3'):
        #    param.requires_grad = True  # Train
        #else:
        #    param.requires_grad = False # Freeze
            
        param_numpy = param.detach().cpu().numpy()
        n_param += len(param_numpy.reshape(-1))
#         print ("[%d] name:[%s] shape:[%s]."%(p_idx,param_name,param_numpy.shape))
#         print ("    val:%s"%(param_numpy.reshape(-1)[:5]))
#         print(f"    --[{param.requires_grad}].")
# print ("Total number of parameters:[%s]."%(format(n_param,',d')))

In [24]:
best_val_acc = 0
best_val_loss = np.inf
patience = 10
cur_count = 0

f1 = F1Score(num_classes=class_num, average='macro').to(device)
best_f1_score = 0

for epoch in range(NUM_EPOCH):
    model.train()
    loss_value = 0
    matches = 0
    for train_batch in tqdm(train_dataloader_mask):
        inputs, labels = train_batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        outs = model(inputs)
        preds = torch.argmax(outs, dim=-1)
        loss = criterion(outs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        torch.save(model, '../checkpoint/resnet50_crop_detection/checkpoint_ep_%d.pth'% epoch)
        
        loss_value += loss.item()
        matches += (preds == labels).sum().item()
        
    train_loss = loss_value / len(train_dataloader_mask)
    train_acc = matches / len(mask_train_set)
    
    print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.3f}, training accuracy {train_acc:.3f}")
    
    with torch.no_grad():
        model.eval()
        val_loss_items = []
        val_acc_items = []
        f1_score = 0
        
        for val_batch in tqdm(val_dataloader_mask):
            inputs, labels = val_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            
            loss_item = criterion(outs, labels).item()
            acc_item = (labels==preds).sum().item()
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            f1_score += f1(outs, labels)
            
        val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
        val_acc = np.sum(val_acc_items) / len(mask_val_set)

        f1_score /= len(val_dataloader_mask)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
        if f1_score > best_f1_score:
            best_f1_score = f1_score
            cur_count = 0
            torch.save(model, '../checkpoint/resnet50_crop_detection/checkpoint_best.pth')
            print("Update checkpoint!!!")
        
        else:
            cur_count += 1
            if cur_count >= patience:
                print("Early Stopping!")
                break

            
        print(f"[val] acc : {val_acc:.3f}, loss : {val_loss:.3f}, f1 score: {f1_score:.3f} -- size({len(val_dataloader_mask)})")
        print(f"best acc : {best_val_acc:.3f}, best loss : {best_val_loss:.3f}, best f1 : {best_f1_score:.3f}")

  0%|          | 0/60 [00:00<?, ?it/s]

epoch[0/50] training loss 1.352, training accuracy 0.576


  0%|          | 0/236 [00:00<?, ?it/s]

Update checkpoint!!!
[val] acc : 0.282, loss : 2.103, f1 score: 0.182 -- size(60)
best acc : 0.282, best loss : 2.103, best f1 : 0.182


  0%|          | 0/60 [00:00<?, ?it/s]

...

[val] acc : 0.890, loss : 0.520, f1 score: 0.805 -- size(60)
best acc : 0.890, best loss : 0.519, best f1 : 0.807


 14%|█▍        | 33/236 [00:34<03:32,  1.05s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

  0%|          | 0/60 [00:00<?, ?it/s]

epoch[49/50] training loss 0.000, training accuracy 0.999


100%|██████████| 60/60 [00:12<00:00,  4.92it/s]

[val] acc : 0.898, loss : 0.575, f1 score: 0.816 -- size(60)
best acc : 0.898, best loss : 0.519, best f1 : 0.817


In [27]:
print(train_dataloader_mask)

In [27]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/new_images/'

model = torch.load('/opt/ml/checkpoint/resnet50_crop_detection/checkpoint_best.pth')
submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [28]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [29]:
image_paths[0]

'/opt/ml/input/data/new_images/cbc5c6e168e63498590db46022617123f1fe1268.jpg'

In [31]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [32]:
batch_size = 64

In [33]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:

    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission_detection_crop2.csv'), index=False)
print('test inference is done!')

test inference is done!
